In [4]:
#구글드라이브 연동
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
# gpu 켜기
import torch
device = torch.device("cuda:0")

In [6]:
# 저장 경로 미리 지정
path = '/content/drive/MyDrive/nlp_c/'

# **로그파일 연동 시키기**

In [7]:
import logging

def make_logger(name=None):
    #1 logger instance를 만든다.
    logger = logging.getLogger(name)

    #2 logger의 level을 가장 낮은 수준인 DEBUG로 설정해둔다.
    logger.setLevel(logging.DEBUG)

    #3 formatter 지정
    formatter = logging.Formatter("%(asctime)s - %(name)s - %(levelname)s - %(message)s")
    
    #4 handler instance 생성
    console = logging.StreamHandler()
    file_handler = logging.FileHandler(filename=path + "logs/correct_final.log",
                                       encoding = 'utf-8')
    
    #5 handler 별로 다른 level 설정
    console.setLevel(logging.INFO)
    file_handler.setLevel(logging.DEBUG)

    #6 handler 출력 format 지정
    console.setFormatter(formatter)
    file_handler.setFormatter(formatter)

    #7 logger에 handler 추가
    logger.addHandler(console)
    logger.addHandler(file_handler)

    return logger

In [8]:
logger = make_logger()

logger.debug("debug logging")
logger.info("info logging")
logger.warning("warning logging")
logger.error("error logging")
logger.critical("critical logging")

2022-04-12 17:21:52,192 - root - INFO - info logging
2022-04-12 17:21:52,194 - root - WARNING - warning logging
2022-04-12 17:21:52,195 - root - ERROR - error logging
2022-04-12 17:21:52,198 - root - CRITICAL - critical logging


# **필요한 환경 다운 및 구축**

## 학습모델 패키지 다운 및 구축(KoBERT)

In [9]:
#깃허브에서 KoBERT 파일 로드
!pip install ipywidgets  # for vscode
!pip install git+https://git@github.com/SKTBrain/KoBERT.git@master

  Cloning https://****@github.com/SKTBrain/KoBERT.git (to revision master) to /tmp/pip-req-build-t4wud7uv
  Running command git clone -q 'https://****@github.com/SKTBrain/KoBERT.git' /tmp/pip-req-build-t4wud7uv
     |████████████████████████████████| 132 kB 4.1 MB/s 
     |████████████████████████████████| 344 kB 33.4 MB/s 
     |████████████████████████████████| 47.3 MB 1.1 MB/s 
     |████████████████████████████████| 4.5 MB 79.4 MB/s 
     |████████████████████████████████| 1.2 MB 59.6 MB/s 
     |████████████████████████████████| 4.0 MB 49.4 MB/s 
     |████████████████████████████████| 77 kB 8.6 MB/s 
     |████████████████████████████████| 6.5 MB 56.7 MB/s 
     |████████████████████████████████| 895 kB 81.6 MB/s 
     |████████████████████████████████| 596 kB 79.5 MB/s 
     |████████████████████████████████| 8.7 MB 51.5 MB/s 
     |████████████████████████████████| 79 kB 11.4 MB/s 
     |████████████████████████████████| 127 kB 91.6 MB/s 
  Created wheel for kobert: filename=ko

In [10]:
# 필요한 모듈 로딩
import pandas as pd
import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import gluonnlp as nlp
import numpy as np
from tqdm.notebook import tqdm
from tqdm import tqdm_notebook

In [11]:
#kobert
from kobert.utils import get_tokenizer
from kobert.pytorch_kobert import get_pytorch_kobert_model

#transformers
from transformers import AdamW
from transformers.optimization import get_cosine_schedule_with_warmup

In [12]:
#BERT 모델, Vocabulary 불러오기
bertmodel, vocab = get_pytorch_kobert_model()

/content/.cache/kobert_v1.zip[██████████████████████████████████████████████████]
/content/.cache/kobert_news_wiki_ko_cased-1087f8699e.spiece[██████████████████████████████████████████████████]


# **맞춤법 검사한 데이터 불러온 뒤 전처리(EDA 방법 포함)**

In [13]:
# 맞춤법 검사 완료한 train 파일 불러오기
train_d = pd.read_csv(path + 'final_data/' + 'correct_train_fin.csv', encoding = 'utf-8-sig')
print(len(train_d))

705530


In [14]:
# index 변경 -> 아래 모델 생성에서 index 오류 발생하여 0부터 re-indexing
# predict 값 추출하고, 다시 원래 y 값으로 변환해주는 방식으로 해야할 듯
# 소분류 dictionary
y_dict = pd.DataFrame({'origin_y' : train_d['y'].unique()}).sort_values(by = 'origin_y')
y_dict['y'] = np.arange(0, len(y_dict))
y_dict = y_dict.astype('str')
s_dict0 = y_dict.set_index('origin_y').to_dict()['y'] # 처음 y값을 모델 train 을 위해 re-indexing
s_dict1 = y_dict.set_index('y').to_dict()['origin_y'] # 뒤에 예측값 다시 y 값으로 return 할 때 사용

In [15]:
# kobert 모델 학습을 위해 reindexing 한 dictionary 저장 -> 후에 모델 예측값 도출 후, 기존 y값으로 되돌리기 위함
import pickle
with open(path+ 'final_data/' + 's_dictionary', 'wb') as f:
    pickle.dump(s_dict1, f)

In [16]:
train_d['y_s'] = train_d['y'].astype('str')
train_d['label_s'] = train_d['y_s'].map(s_dict0)

## EDA 부분

In [17]:
# text augmentation
# pip install -U nltk
import nltk; 
nltk.download('omw-1.4');
# nltk.download('wordnet') # 영문 버전

[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Unzipping corpora/omw-1.4.zip.


In [18]:
# eda 폴더 생성
% cd /content/drive/MyDrive/nlp_c
# !git clone https://github.com/jasonwei20/eda_nlp
# !git clone https://github.com/catSirup/KorEDA
# eda는 eda_nlp/code 폴더에, wordnet.pickle 은 eda_nlp 폴더로 이동시키고, 진행
% cd eda_nlp/
# 추가적으로 augment.py 64번째 항에 eda -> EDA로 변경해야 실행됨

/content/drive/MyDrive/nlp_c
/content/drive/MyDrive/nlp_c/eda_nlp


In [19]:
s_class_n = pd.DataFrame(train_d['y_s'].value_counts().sort_values())
# s_class_n.to_csv(path + 'testset_class.csv', index=False, encoding='EUC-KR')
s_class = s_class_n[s_class_n['y_s'] < 500].index.tolist()
len(s_class)

85

In [20]:
# n수가 부족한 class aug_num 차등으로 증강(적은 순서대로 20, 10 ,5) -> 상대적으로 부족한 클래스 데이터 비율이 더 높아지는 것을 조금이라도 방지하고자 함
s_class1 = s_class[:30]
s_class2 = s_class[30:60]
s_class3 = s_class[60:]

In [21]:
few_d1 = train_d[train_d['y_s'].isin(s_class1)]
few_d2 = train_d[train_d['y_s'].isin(s_class2)]
few_d3 = train_d[train_d['y_s'].isin(s_class3)]

In [22]:
# n이 100개 이하인 클래스 뽑아서 augmentation 가능한 파일 형태로 만들어주기
txt_aug_list = [str(a) + '\t' + str(b) for a, b in zip(few_d1['label_s'], few_d1['clean_done'])]
with open(path + 'final_data/' + 'text_aug_1.txt', 'w') as f:
  f.write('\n'.join(txt_aug_list) + '\n')

txt_aug_list = [str(a) + '\t' + str(b) for a, b in zip(few_d2['label_s'], few_d2['clean_done'])]
with open(path + 'final_data/' + 'text_aug_2.txt', 'w') as f:
  f.write('\n'.join(txt_aug_list) + '\n')

txt_aug_list = [str(a) + '\t' + str(b) for a, b in zip(few_d3['label_s'], few_d3['clean_done'])]
with open(path + 'final_data/' + 'text_aug_3.txt', 'w') as f:
  f.write('\n'.join(txt_aug_list) + '\n')

# input file 형식 -> txt 파일 내 한 행 당 label + \t + text 형태로 들어간 파일 
# SR: Synonym Replacement, 특정 단어를 유의어로 교체
# RI: Random Insertion, 임의의 단어를 삽입
# RS: Random Swap, 문장 내 임의의 두 단어의 위치를 바꿈
# RD: Random Deletion: 임의의 단어를 삭제
!python code/augment.py --input=/content/drive/MyDrive/nlp_c/final_data/text_aug_1.txt --output=/content/drive/MyDrive/nlp_c/final_data/test_aug_eda_1.txt --num_aug=20 --alpha_sr=0.1 --alpha_rd=0.2 --alpha_ri=0.1 --alpha_rs=0.0
!python code/augment.py --input=/content/drive/MyDrive/nlp_c/final_data/text_aug_2.txt --output=/content/drive/MyDrive/nlp_c/final_data/test_aug_eda_2.txt --num_aug=10 --alpha_sr=0.1 --alpha_rd=0.2 --alpha_ri=0.1 --alpha_rs=0.0
!python code/augment.py --input=/content/drive/MyDrive/nlp_c/final_data/text_aug_3.txt --output=/content/drive/MyDrive/nlp_c/final_data/test_aug_eda_3.txt --num_aug=5 --alpha_sr=0.1 --alpha_rd=0.2 --alpha_ri=0.1 --alpha_rs=0.0

generated augmented sentences with eda for /content/drive/MyDrive/nlp_c/final_data/text_aug_1.txt to /content/drive/MyDrive/nlp_c/final_data/test_aug_eda_1.txt with num_aug=20
generated augmented sentences with eda for /content/drive/MyDrive/nlp_c/final_data/text_aug_2.txt to /content/drive/MyDrive/nlp_c/final_data/test_aug_eda_2.txt with num_aug=10
generated augmented sentences with eda for /content/drive/MyDrive/nlp_c/final_data/text_aug_3.txt to /content/drive/MyDrive/nlp_c/final_data/test_aug_eda_3.txt with num_aug=5


In [23]:
# augmentation 완료한 데이터 불러와서 기존데이터셋에 붙여주기(augmentation 대상 데이터는 삭제)
with open('/content/drive/MyDrive/nlp_c/final_data/test_aug_eda_1.txt', "r") as file:
  strings = file.readlines()
aug_d1 = pd.DataFrame([x.split('\n')[0].split('\t') for x in strings])
aug_d1.columns = ['label_s', 'clean_done']

with open('/content/drive/MyDrive/nlp_c/final_data/test_aug_eda_2.txt', "r") as file:
  strings = file.readlines()
aug_d2 = pd.DataFrame([x.split('\n')[0].split('\t') for x in strings])
aug_d2.columns = ['label_s', 'clean_done']

with open('/content/drive/MyDrive/nlp_c/final_data/test_aug_eda_3.txt', "r") as file:
  strings = file.readlines()
aug_d3 = pd.DataFrame([x.split('\n')[0].split('\t') for x in strings])
aug_d3.columns = ['label_s', 'clean_done']

train_d = train_d[train_d['y_s'].isin(s_class)==False]
sample_d = pd.concat([train_d[['label_s', 'clean_done']], aug_d1, aug_d2, aug_d3], axis = 0, ignore_index = True)

In [24]:
sample_d['len'] = sample_d['clean_done'].astype(str).apply(len)

In [25]:
i = 798000
sample_d[i:i+50]

,label_s,clean_done,len
798000,139,당진화력 예인선 현장에서 내 기타 해상 운수업,25
798001,139,당진화력 내 현장에서 예인선 기타 해상 운수업,25
798002,103,각종 건설현장 폐기물을 중간처리,17
798003,103,각종 건설현장 폐기물을 중간처리,17
798004,103,각종 건설현장 폐기물을 중간처리,17
798005,103,각종 건설현장 폐기물을,12
798006,103,건설현장 각종 폐기물을 중간처리,17
798007,103,각종 건설현장 폐기물을 중간처리,17
798008,196,환경 관리사업소에서 하수 및 수행하여 폐기물 환경보존 행정,32
798009,196,환경 관리사업소에서 하수 폐기물 및 위생처리를 수행하여 폐기물 환경보존 경,41


# **처리한 데이터 kobert 모델에 학습**

In [26]:
# train & test set 나누기
from sklearn.model_selection import train_test_split
# dataset_train, dataset_test = train_test_split(train_d, test_size=0.2, shuffle=True, random_state=30)
# dataset_test.to_csv('/content/drive/MyDrive/nlp_c/testset.csv', index=False, encoding = 'EUC-KR')

# stratify 를 target으로 지정해 비율을 맞춤으로써, 성능향상 가능
# but, 현재 target 변수 class 비율의 불균형으로 오류 발생(1,2 개짜리 class 다수 존재)
dataset_train, dataset_test, y_train, y_test = train_test_split(sample_d['clean_done'],
                               sample_d['label_s'], random_state=132, stratify=sample_d['label_s']) 
# 모델 검증용 미리 뽑아놓기
dataset_test.to_csv(path + 'final_data/' + 'testset.csv', index=False, encoding = 'utf-8-sig')
y_test.to_csv(path + 'final_data/' + 'testset_y.csv', index=False, encoding = 'utf-8-sig')

In [27]:
dataset_train = [[str(a), str(b)] for a, b in zip(dataset_train, y_train)]
dataset_test = [[str(a), str(b)] for a, b in zip(dataset_test, y_test)]

In [28]:
print(len(dataset_train))
print(len(dataset_test))

614287
204763


In [29]:
print(dataset_train[:10])
print(dataset_test[:10])

[['사업장에서 의뢰를 받아 철근 및 콘크리트 공사업', '108'], ['실내장식 집에서 벽지 장판 등을 이용하여 벽지 장판 판매 시공', '111'], ['두리 수산에서 일반 소비자에게 소매 각종 생선 팬 맥', '126'], ['내과 외래환자 내과 진료', '208'], ['부동산 중개소 중개 및 계약을 체결 수수료 받음 부동산 중개 서비스', '169'], ['건축 설계 및 관련 서비스업  건축설계', '178'], ['부동산 중개소에서 계약 및 중개에 의해 수수료 받음 부동산 거래 중개서비스', '169'], ['커피와 음료 판매  ', '147'], ['솜 원료를 중합  방사 ', '48'], ['사업장에서 일반인 방문객을 통하여 명상수련 서비스', '205']]
[['신발가게에서 일반 소비자에게 소매 남녀 정장화', '128'], ['철 입고  표면가공 ', '63'], ['상가 관리사무소에서 상가 입주민을 위해 상가 관리 서비스', '169'], ['정육점에서 소비자를 대상으로 소고기 돼지고기', '126'], ['사무실에서 의뢰를 받아 세무대행', '173'], ['강습짱에서 일반 고객 대상으로 필라테스 강습', '205'], ['모터  감속기 조립  도장 ', '81'], ['산업 사용자에게 증기', '99'], ['모텔 고객 숙박 서비스', '145'], ['정육점에서 일반 소비자에게 소고기  돼지고기 판매', '147']]


In [30]:
# KoBERT 입력 데이터로 만들기
# BERT 모델에 들어가기 위한 dataset을 만들어주는 클래스
class BERTDataset(Dataset):
    def __init__(self, dataset, sent_idx, label_idx, bert_tokenizer, max_len,
                 pad, pair):
        transform = nlp.data.BERTSentenceTransform(
            bert_tokenizer, max_seq_length=max_len, pad=pad, pair=pair)

        self.sentences = [transform([i[sent_idx]]) for i in dataset]
        self.labels = [np.int32(i[label_idx]) for i in dataset]

    def __getitem__(self, i):
        return (self.sentences[i] + (self.labels[i], ))

    def __len__(self):
        return (len(self.labels))

In [31]:
# Setting parameters
# 이건 나중에 최적화 값 찾아봐야할 듯
max_len = 64
batch_size = 128
warmup_ratio = 0.1
num_epochs = 5
max_grad_norm = 1
log_interval = 200
learning_rate = 5e-5 # 0.0001 # 

# 토큰화 실행
tokenizer = get_tokenizer()
tok = nlp.data.BERTSPTokenizer(tokenizer, vocab, lower=False)

using cached model. /content/drive/MyDrive/nlp_c/eda_nlp/.cache/kobert_news_wiki_ko_cased-1087f8699e.spiece


In [32]:
data_train = BERTDataset(dataset_train, 0, 1, tok, max_len, pad = True, pair = False)
data_test = BERTDataset(dataset_test, 0, 1, tok, max_len, pad = True, pair = False)

In [33]:
sentence = dataset_train[30][0] 
print(sentence)
print(tok(sentence))

# 토큰화 패딩 처리 후 결과값 
print(data_train[30])

음식점에서 객석을 갖추고 한식 판매  
['▁음식', '점', '에서', '▁', '객', '석', '을', '▁갖추', '고', '▁한', '식', '▁판매']
(array([   2, 3609, 7220, 6903,  517, 5370, 6557, 7088,  827, 5439, 4955,
       6730, 4809,    3,    1,    1,    1,    1,    1,    1,    1,    1,
          1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
          1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
          1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
          1,    1,    1,    1,    1,    1,    1,    1,    1], dtype=int32), array(14, dtype=int32), array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
      dtype=int32), 147)


In [34]:
# torch 형식의 dataset 생성
# num_worker 은 gpu 활정화 정도, 5로 하니 오히려 과부화가 걸려 4로 조정
train_dataloader = torch.utils.data.DataLoader(data_train, batch_size=batch_size, num_workers=4)
test_dataloader = torch.utils.data.DataLoader(data_test, batch_size=batch_size, num_workers=4)

In [35]:
# 클래스 수 조정
print(len(y_dict))

225


In [36]:
# KoBERT 학습모델 만들기
class BERTClassifier(nn.Module):
    def __init__(self,
                 bert,
                 hidden_size = 768,
                 num_classes=len(y_dict),   ##클래스 수 조정해줘야함##
                 dr_rate=None,
                 params=None):
        super(BERTClassifier, self).__init__()
        self.bert = bert
        self.dr_rate = dr_rate
                 
        self.classifier = nn.Linear(hidden_size , num_classes)
        if dr_rate:
            self.dropout = nn.Dropout(p=dr_rate)
    
    def gen_attention_mask(self, token_ids, valid_length):
        attention_mask = torch.zeros_like(token_ids)
        for i, v in enumerate(valid_length):
            attention_mask[i][:v] = 1
        return attention_mask.float()

    def forward(self, token_ids, valid_length, segment_ids):
        attention_mask = self.gen_attention_mask(token_ids, valid_length)
        
        _, pooler = self.bert(input_ids = token_ids, token_type_ids = segment_ids.long(), attention_mask = attention_mask.float().to(token_ids.device))
        if self.dr_rate:
            out = self.dropout(pooler)
        return self.classifier(out)

In [ ]:
# GPU 실행 오류 나면 사용
# import os
# os.environ['CUDA_LAUNCH_BLOCKING'] = "1"
# os.environ["CUDA_VISIBLE_DEVICES"] = "0"

In [37]:
model = BERTClassifier(bertmodel, dr_rate=0.5).to(device)

In [38]:
# Prepare optimizer and schedule (linear warmup and decay)
no_decay = ['bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
    {'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
]

In [39]:
optimizer = AdamW(optimizer_grouped_parameters, lr=learning_rate)
loss_fn = nn.CrossEntropyLoss()

/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,


In [40]:
t_total = len(train_dataloader) * num_epochs
warmup_step = int(t_total * warmup_ratio)

In [41]:
scheduler = get_cosine_schedule_with_warmup(optimizer, num_warmup_steps=warmup_step, num_training_steps=t_total)

In [42]:
def calc_accuracy(X,Y):
    max_vals, max_indices = torch.max(X, 1)
    train_acc = (max_indices == Y).sum().data.cpu().numpy()/max_indices.size()[0]
    return train_acc

In [ ]:
# pip install torch==1.8.0+cu111 torchvision==0.9.0+cu111 torchaudio==0.8.0 -f https://download.pytorch.org/whl/torch_stable.html

In [43]:
highest_acc = 0
patience = 0

# 최종 모델 학습시키기
for e in range(num_epochs):
    train_acc = 0.0
    test_acc = 0.0
    model.train()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(train_dataloader)):
        optimizer.zero_grad()
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        loss = loss_fn(out, label)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)
        optimizer.step()
        scheduler.step()  # Update learning rate schedule
        train_acc += calc_accuracy(out, label)
        if batch_id % log_interval == 0:
            print("epoch {} batch id {} loss {} train acc {}".format(e+1, batch_id+1, loss.data.cpu().numpy(), train_acc / (batch_id+1)))
    print("epoch {} train acc {}".format(e+1, train_acc / (batch_id+1)))
    
    model.eval()

    for test_batch_id, (test_token_ids, test_valid_length, test_segment_ids, test_label) in enumerate(tqdm_notebook(test_dataloader)):
        test_token_ids = test_token_ids.long().to(device)
        test_segment_ids = test_segment_ids.long().to(device)
        test_valid_length= test_valid_length
        test_label = test_label.long().to(device)
        test_out = model(token_ids, valid_length, segment_ids)
        test_loss = loss_fn(out, label)
        test_acc += calc_accuracy(out, label)
    print("epoch {} test acc {}".format(e+1, test_acc / (test_batch_id+1)))

    if test_acc > highest_acc:
        torch.save({
            'epoch': e,
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'loss': test_loss,
            }, path + 'final_data/' + 'correct_model_fin.pt')
        patience = 0
    else:
        print("test acc did not improved. best:{} current:{}".format(highest_acc, test_acc))
        patience += 1
        if patience > 5:
            break
    print('current patience: {}'.format(patience))
    print("************************************************************************************")

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  if __name__ == '__main__':


  0%|          | 0/4800 [00:00<?, ?it/s]

epoch 1 batch id 1 loss 5.470526218414307 train acc 0.0
epoch 1 batch id 201 loss 5.183148384094238 train acc 0.04287157960199005
epoch 1 batch id 401 loss 4.4437031745910645 train acc 0.08350218204488778
epoch 1 batch id 601 loss 3.7217705249786377 train acc 0.13737520798668884
epoch 1 batch id 801 loss 3.2203402519226074 train acc 0.19774110486891386
epoch 1 batch id 1001 loss 2.5548582077026367 train acc 0.2525131118881119
epoch 1 batch id 1201 loss 2.060025215148926 train acc 0.3013049021648626
epoch 1 batch id 1401 loss 1.7654931545257568 train acc 0.3442184154175589
epoch 1 batch id 1601 loss 1.7790783643722534 train acc 0.3823538023110556
epoch 1 batch id 1801 loss 1.3805230855941772 train acc 0.415585091615769
epoch 1 batch id 2001 loss 1.0565495491027832 train acc 0.445050912043978
epoch 1 batch id 2201 loss 1.0683343410491943 train acc 0.4712275670149932
epoch 1 batch id 2401 loss 1.3318103551864624 train acc 0.49479708975426906
epoch 1 batch id 2601 loss 0.9846600294113159 t

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:28: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`


  0%|          | 0/1600 [00:00<?, ?it/s]

...

epoch 4 test acc 0.9333333333333352
current patience: 0
************************************************************************************


  0%|          | 0/4800 [00:00<?, ?it/s]

epoch 5 batch id 1 loss 0.2737545073032379 train acc 0.9140625
epoch 5 batch id 201 loss 0.2553758919239044 train acc 0.9324860074626866
epoch 5 batch id 401 loss 0.2817900776863098 train acc 0.9322007481296758
epoch 5 batch id 601 loss 0.2567768692970276 train acc 0.9328462978369384
epoch 5 batch id 801 loss 0.2468549758195877 train acc 0.9336278870162297
epoch 5 batch id 1001 loss 0.22222311794757843 train acc 0.9333791208791209
epoch 5 batch id 1201 loss 0.25492382049560547 train acc 0.9332717527060783
epoch 5 batch id 1401 loss 0.15025579929351807 train acc 0.9334515524625268
epoch 5 batch id 1601 loss 0.22277267277240753 train acc 0.933493714865709
epoch 5 batch id 1801 loss 0.19275978207588196 train acc 0.9338171501943365
epoch 5 batch id 2001 loss 0.22030514478683472 train acc 0.9345483508245878
epoch 5 batch id 2201 loss 0.21622973680496216 train acc 0.9348627044525216
epoch 5 batch id 2401 loss 0.3563814163208008 train acc 0.9350205643481883
epoch 5 batch id 2601 loss 0.227429

  0%|          | 0/1600 [00:00<?, ?it/s]

epoch 5 test acc 1.0
current patience: 0
************************************************************************************


In [ ]:
# # 학습한 모델 pickle 형태로 저장

# import pickle
# # path = '/content/drive/MyDrive/nlp_c/'

# with open(path+'model_trial_fin_noclean.pickle', 'wb') as f:
#     pickle.dump(model, f)

# **코드북으로 제출형식 만들기**

## 한국표준산업분류(10차)_국문 자료 이용해서 코드 북 만들기

In [44]:
import numpy as np 
import pandas as pd

In [45]:
path = '/content/drive/MyDrive/nlp_c/'
code_book = pd.read_excel(path + '한국표준산업분류(10차)_국문.xlsx', header = 1)

In [46]:
code_book = code_book.dropna(subset = ['소분류(232)'])
code_book[:10]

,대분류(21),Unnamed: 1,중분류(77),Unnamed: 3,소분류(232),Unnamed: 5,세분류(495),Unnamed: 7,"세세분류(1,196)",Unnamed: 9
0,코드,항목명,코드,항목명,코드,항목명,코드,항목명,코드,항목명
1,A,"농업, 임업 및 어업(01~03)",01,농업,011,작물 재배업,0111,곡물 및 기타 식량작물 재배업,01110,곡물 및 기타 식량작물 재배업
11,NaN,NaN,NaN,NaN,012,축산업,0121,소 사육업,01211,젖소 사육업
18,NaN,NaN,NaN,NaN,013,작물재배 및 축산 복합농업,0130,작물재배 및 축산 복합농업,01300,작물재배 및 축산 복합농업
19,NaN,NaN,NaN,NaN,014,작물재배 및 축산 관련 서비스업,0141,작물재배 관련 서비스업,01411,작물재배 지원 서비스업
22,NaN,NaN,NaN,NaN,015,수렵 및 관련 서비스업,0150,수렵 및 관련 서비스업,01500,수렵 및 관련 서비스업
23,NaN,NaN,02,임업,020,임업,0201,영림업,02011,임업용 종묘 생산업
28,NaN,NaN,03,어업,031,어로 어업,0311,해수면 어업,03111,원양 어업
31,NaN,NaN,NaN,NaN,032,양식어업 및 어업관련 서비스업,0321,양식 어업,03211,해수면 양식 어업
35,B,광업(05~08),05,"석탄, 원유 및 천연가스 광업",051,석탄 광업,0510,석탄 광업,05100,석탄 광업


In [47]:
code = code_book[['대분류(21)', '중분류(77)', '소분류(232)', 'Unnamed: 5']][1:].reset_index(drop=True)

In [48]:
def na_to_code(data):
  data_l = []
  temp = data[0]

  for i in range(0, len(data)):
    if pd.isna(data[i]):
      data[i] =  temp
    else:
      temp = data[i] 
    data_l.append(temp)
  return data_l

In [49]:
big = na_to_code(code.iloc[:,0].tolist())
middle = na_to_code(code.iloc[:,1].tolist())
small = na_to_code(code.iloc[:,2].tolist())

In [50]:
code_b = pd.DataFrame(zip(big,middle,small), columns = ['big', 'middle', 'small'])
code_b['y'] = code_b['small'].astype('int64')
code_b['name'] = code['Unnamed: 5']
code_b
code_b.to_excel(path + 'codebook_dict.xlsx', index=False, encoding = 'EUC-KR')

In [51]:
code_b = pd.read_excel(path + 'codebook_dict.xlsx', dtype = {'big': str, 'middle': str, 'small': str})

In [52]:
code_b = code_b.iloc[:,:-1] # name은 참고용이므로 제외
dict_fin = code_b.set_index('y').T.to_dict('list') # 소분류값을 key 로 한 dictionary

## 학습한 모델 불러오기 및 환경 구축

In [17]:
# # train 할 때, 메모리를 많이 사용하여, 비우기
# import gc
# gc.collect()
# del model
# torch.cuda.empty_cache()

In [18]:
# test 하기전 모델 기본 값 불러오기

# KoBERT 입력 데이터로 만들기
# BERT 모델에 들어가기 위한 dataset을 만들어주는 클래스
class BERTDataset(Dataset):
    def __init__(self, dataset, sent_idx, label_idx, bert_tokenizer, max_len,
                 pad, pair):
        transform = nlp.data.BERTSentenceTransform(
            bert_tokenizer, max_seq_length=max_len, pad=pad, pair=pair)

        self.sentences = [transform([i[sent_idx]]) for i in dataset]
        self.labels = [np.int32(i[label_idx]) for i in dataset]

    def __getitem__(self, i):
        return (self.sentences[i] + (self.labels[i], ))

    def __len__(self):
        return (len(self.labels))

# 토큰화 실행
tokenizer = get_tokenizer()
tok = nlp.data.BERTSPTokenizer(tokenizer, vocab, lower=False)

# KoBERT 학습모델 만들기
class BERTClassifier(nn.Module):
    def __init__(self,
                 bert,
                 hidden_size = 768,
                 num_classes=225,   ##클래스 수 조정해줘야함##
                 dr_rate=None,
                 params=None):
        super(BERTClassifier, self).__init__()
        self.bert = bert
        self.dr_rate = dr_rate
                 
        self.classifier = nn.Linear(hidden_size , num_classes)
        if dr_rate:
            self.dropout = nn.Dropout(p=dr_rate)
    
    def gen_attention_mask(self, token_ids, valid_length):
        attention_mask = torch.zeros_like(token_ids)
        for i, v in enumerate(valid_length):
            attention_mask[i][:v] = 1
        return attention_mask.float()

    def forward(self, token_ids, valid_length, segment_ids):
        attention_mask = self.gen_attention_mask(token_ids, valid_length)
        
        _, pooler = self.bert(input_ids = token_ids, token_type_ids = segment_ids.long(), attention_mask = attention_mask.float().to(token_ids.device))
        if self.dr_rate:
            out = self.dropout(pooler)
        return self.classifier(out)

using cached model. /content/.cache/kobert_news_wiki_ko_cased-1087f8699e.spiece


In [19]:
# import pickle
# # 학습한 model 열기

# with open(path+'model_trial_fin_noclean.pickle', 'rb') as f:
#     model = pickle.load(f)

In [20]:
# Setting parameters
# 이건 나중에 최적화 값 찾아봐야할 듯
max_len = 64
batch_size = 128
warmup_ratio = 0.1
num_epochs = 6
max_grad_norm = 1
log_interval = 200
learning_rate = 5e-5 # 0.0001 # 

In [21]:
model = BERTClassifier(bertmodel, dr_rate=0.5).to(device)
# Prepare optimizer and schedule (linear warmup and decay)
no_decay = ['bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
    {'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
]
optimizer = AdamW(optimizer_grouped_parameters, lr=learning_rate)

checkpoint = torch.load(path + 'final_data/' + 'correct_model_fin.pt') # 학습한 파일 경로 지정
model.load_state_dict(checkpoint['model_state_dict'])
optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
epoch = checkpoint['epoch']
loss = checkpoint['loss']

/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,


In [58]:
model.eval()

BERTClassifier(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(8002, 768, padding_idx=1)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True

## 맞춤법 처리한 제출파일 제출 형식으로 바꾸기

In [59]:
test = pd.read_csv(path + 'final_data/' + 'correct_sub_fin.csv', encoding = 'utf-8-sig')

In [60]:
dataset_test = [[str(a), '0'] for a in test['clean_done']]
dataset_test[:20]

[['치킨전문점에서 고객의 주문에 의해 치킨 판매', '0'],
 ['산업공구 다른 소매업자에게 철물 수공구', '0'],
 ['절에서 신도를 대상으로 불교단체 운영', '0'],
 ['영업장에서 고객 요구로 자동차 튜닝', '0'],
 ['실내포장마차에서 접객시설을 갖추고 소주 맥주 제공', '0'],
 ['철 아크릴 코맥스 스크린인쇄 명판', '0'],
 ['음식점 접객시설 가지고 조 개구 이 판매', '0'],
 ['스테인리스를 프레스가공하여 제조 주방용품', '0'],
 ['수리 서비스센터에서 전문수리 수입차', '0'],
 ['약품 화공   미싱 완성품 입고  수선 ', '0'],
 ['밀가루  쇼트닝 원재료 입고  반죽 ', '0'],
 ['이발소에서 일반인 대상으로 고객의 두발을 손질함', '0'],
 ['목재 고객의 요구에 따라 무대장치', '0'],
 ['의원에서 소아 청소년을 대상으로 진료서비스', '0'],
 ['산업공단 조성 시 의뢰받아 사무실에서 측량 토목설계', '0'],
 ['태국 전통 마사지숍 일반 고객 대상 마사지', '0'],
 ['사업장에서 관련 사용자에게 선박엔진부품', '0'],
 ['상가에서 주문에 의해 반찬 도시락', '0'],
 ['문구용품에서 일반인 대상 문구류 사무용품 소매', '0'],
 ['엔지니어링 및 건축 관련 고객 요청에 의해 도면 설계', '0']]

In [61]:
from tqdm.notebook import tqdm

In [62]:
# 예측 함수 생성
# Setting parameters
# train 학습 모델 설정할 때와 동일하게 설정
def predict_set(dataset_test):

    test_acc = 0.0

    tokenizer = get_tokenizer()
    tok = nlp.data.BERTSPTokenizer(tokenizer, vocab, lower=False)

    data_test = BERTDataset(dataset_test, 0, 1, tok, max_len, True, False)

    test_dataloader = torch.utils.data.DataLoader(data_test, batch_size=batch_size, num_workers=4)

    out_list =[]

    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm(test_dataloader)):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)

        valid_length= valid_length
        label = label.long().to(device)

        out = model(token_ids, valid_length, segment_ids)
        output = out.detach().cpu().tolist()
        out_list.append(output)

    pd = sum(out_list,[])
    pd_list = pd_list = [np.argmax(i) for i in pd]
    return pd_list

In [63]:
p_test = predict_set(dataset_test)

using cached model. /content/drive/MyDrive/nlp_c/eda_nlp/.cache/kobert_news_wiki_ko_cased-1087f8699e.spiece


  0%|          | 0/782 [00:00<?, ?it/s]

In [64]:
import pickle
# 학습하기전 기존 y값 사전 열기
with open(path+'final_data/' + 's_dictionary', 'rb') as f:
    s_dict = pickle.load(f)

In [65]:
test['predict_y'] = p_test
# 1. 모델 학습하기 전 기존 y값 변수로 변환
test['predict_y'] = test['predict_y'].astype('str').map(s_dict) 
cols = ['digit_1', 'digit_2', 'digit_3']
# 2. 코드북에서 소분류를 통해 대/중분류 함께 예측
test[cols] = test['predict_y'].astype('int64').map(dict_fin).apply(lambda x: pd.Series(x))

In [66]:
test_fin = test[['AI_id', 'digit_1', 'digit_2', 'digit_3', 'text_obj', 'text_mthd', 'text_deal']]
test_fin.to_csv(path + 'final_data/' + 'submission_fin_0413.csv', index=False, encoding='EUC-KR')

In [67]:
test_fin[:50]

,AI_id,digit_1,digit_2,digit_3,text_obj,text_mthd,text_deal
0,id_000001,I,56,561,치킨전문점에서,고객의주문에의해,치킨판매
1,id_000002,G,46,466,산업공구,다른 소매업자에게,철물 수공구
2,id_000003,S,94,949,절에서,신도을 대상으로,불교단체운영
3,id_000004,S,95,952,영업장에서,고객요구로,자동차튜닝
4,id_000005,I,56,562,실내포장마차에서,접객시설을 갖추고,"소주,맥주제공"
5,id_000006,C,18,181,"철,아크릴,포맥스",스크린인쇄,명판
6,id_000007,I,56,561,음식점,접객시설가지고,조개구이판매
7,id_000008,C,25,259,스테인레스를,프레스가공하여제조,주방용품
8,id_000009,S,95,952,수리,서비스센터에서,전문수리 수입차
9,id_000010,C,25,259,"약품(화공), 미싱","완성품입고, 수선",NaN
